In [ ]:
import wandb
import pandas as pd
import os

OUTPUT_DIR = "./wandb_exports"
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [ ]:
api = wandb.Api()

In [ ]:
ENTITY = "hal2k-n-a"
PROJECT = "lat-interference-analysis"
BASELINE_RUN_ID = "l0t94iae"
LAT_RUN_ID = "q5psuu9y"

In [ ]:
baseline_run = api.run(f"{ENTITY}/{PROJECT}/{BASELINE_RUN_ID}")
lat_run = api.run(f"{ENTITY}/{PROJECT}/{LAT_RUN_ID}")

print(f"Baseline: {baseline_run.name} ({baseline_run.displayName}) - {baseline_run.state}")
print(f"LAT: {lat_run.name} ({lat_run.displayName}) - {lat_run.state}")

In [ ]:
print("Exporting baseline metrics (unsampled)...")
baseline_history = list(baseline_run.scan_history())
baseline_df = pd.DataFrame(baseline_history)
baseline_df.to_csv(f"{OUTPUT_DIR}/baseline_metrics.csv", index=False)
print(f"  Saved {len(baseline_df)} rows to {OUTPUT_DIR}/baseline_metrics.csv")

print("\nExporting LAT metrics (unsampled)...")
lat_history = list(lat_run.scan_history())
lat_df = pd.DataFrame(lat_history)
lat_df.to_csv(f"{OUTPUT_DIR}/lat_metrics.csv", index=False)
print(f"  Saved {len(lat_df)} rows to {OUTPUT_DIR}/lat_metrics.csv")

In [ ]:
import json

baseline_config = dict(baseline_run.config)
lat_config = dict(lat_run.config)

with open(f"{OUTPUT_DIR}/baseline_config.json", "w") as f:
    json.dump(baseline_config, f, indent=2, default=str)
print(f"Saved baseline config to {OUTPUT_DIR}/baseline_config.json")

with open(f"{OUTPUT_DIR}/lat_config.json", "w") as f:
    json.dump(lat_config, f, indent=2, default=str)
print(f"Saved LAT config to {OUTPUT_DIR}/lat_config.json")

In [ ]:
baseline_summary = dict(baseline_run.summary)
lat_summary = dict(lat_run.summary)

with open(f"{OUTPUT_DIR}/baseline_summary.json", "w") as f:
    json.dump(baseline_summary, f, indent=2, default=str)
print(f"Saved baseline summary to {OUTPUT_DIR}/baseline_summary.json")

with open(f"{OUTPUT_DIR}/lat_summary.json", "w") as f:
    json.dump(lat_summary, f, indent=2, default=str)
print(f"Saved LAT summary to {OUTPUT_DIR}/lat_summary.json")

In [ ]:
print("\n" + "="*60)
print("FINAL METRICS COMPARISON")
print("="*60)

metrics = [
    "losses/mse_loss",
    "losses/auxiliary_reconstruction_loss",
    "losses/overall_loss",
    "metrics/explained_variance",
    "metrics/l0",
    "reconstruction_quality/mse",
    "reconstruction_quality/cossim",
    "reconstruction_quality/explained_variance",
    "sparsity/dead_features",
    "model_performance_preservation/ce_loss_score",
]

print(f"{'Metric':<45} {'Baseline':>12} {'LAT':>12} {'Ratio':>10}")
print("-" * 80)

for m in metrics:
    bl_val = baseline_summary.get(m)
    lat_val = lat_summary.get(m)
    if bl_val is not None and lat_val is not None:
        ratio = lat_val / bl_val if bl_val != 0 else float('inf')
        print(f"{m:<45} {bl_val:>12.4f} {lat_val:>12.4f} {ratio:>10.3f}")
    else:
        print(f"{m:<45} {'N/A':>12} {'N/A':>12} {'N/A':>10}")

In [ ]:
print(f"\nExported files:")
for f in os.listdir(OUTPUT_DIR):
    size = os.path.getsize(f"{OUTPUT_DIR}/{f}")
    print(f"  {f}: {size/1024:.1f} KB")